In [71]:
import pandas as pd
import re
import numpy as np
import matplotlib
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [17]:
airbnb = pd.read_csv("data/listings.csv")
airbnb = airbnb[airbnb["neighbourhood_group_cleansed"] == "PORTO"]

## Licenças

In [18]:
def extract_code(text):
    # Use regular expression to find and extract all groups of consecutive numbers
    numbers = re.findall(r'\d+', text)
    
    # Find the largest group of consecutive numbers (if any)
    if numbers:
        largest_number = max(numbers, key=len)
        return largest_number
    return None
airbnb["license"] = airbnb["license"].astype(str)
airbnb['license_cleaned'] = airbnb['license'].apply(extract_code)
airbnb["license_cleaned"] = airbnb["license_cleaned"].replace({"6":"0",
"Registo n.º: 237/2010 (4 de Maio) - Turismo Portugal":"237",
"553/2016":"553"})
airbnb

In [40]:
RNAL = pd.read_csv("../python/final/porto.csv")
RNAL["NrRNAL"] = RNAL["NrRNAL"].astype(str)

#gerado 07/07/2023
RNET = pd.read_csv("../python/final/RNET.csv")
RNET["NrRNET"] = RNET["NrRNET"].astype(str)

In [48]:
airbnb['license_cleaned'].isna().sum()

1333

In [46]:
len(set(airbnb["license_cleaned"]) - set(RNAL["NrRNAL"]) - set(RNET["NrRNET"]))

548

In [45]:
def has_license(license_number):
    if license_number in set(RNAL["NrRNAL"]):
        return 'RNAL'
    elif license_number in set(RNET["NrRNET"]):
        return 'RNET'
    else:
        return 'None'
airbnb['has_license'] = airbnb['license_cleaned'].apply(has_license)
airbnb['has_license'].value_counts()

has_license
RNAL    7387
None    2120
RNET      88
Name: count, dtype: int64

In [97]:
airbnb_with_RNAL = airbnb[airbnb["has_license"] == "RNAL"][["license_cleaned","first_review"]]
airbnb_with_RNAL = airbnb_with_RNAL.rename(columns={"license_cleaned":"NrRNAL"})

merged_airbnb_RNAL = RNAL.merge(airbnb_with_RNAL, on='NrRNAL', how='inner')

merged_airbnb_RNAL.shape

(7387, 17)

In [101]:
merged_airbnb_RNAL = merged_airbnb_RNAL[~merged_airbnb_RNAL["first_review"].isna()]
merged_airbnb_RNAL.shape

(6763, 17)

In [106]:
# Convert date columns to datetime objects
merged_airbnb_RNAL['DataAberturaPublico'] = pd.to_datetime(merged_airbnb_RNAL['DataAberturaPublico'])
merged_airbnb_RNAL['first_review'] = pd.to_datetime(merged_airbnb_RNAL['first_review'])
merged_airbnb_RNAL['first_review'] = merged_airbnb_RNAL['first_review'].dt.tz_localize('UTC')

# Create a new boolean column based on the comparison
merged_airbnb_RNAL['license_date_after_review'] = merged_airbnb_RNAL['DataAberturaPublico'] > merged_airbnb_RNAL['first_review']

In [108]:
merged_airbnb_RNAL[["DataAberturaPublico", "first_review", "license_date_after_review"]]

,DataAberturaPublico,first_review,license_date_after_review
1,2014-12-09 00:00:00+00:00,2014-04-09 00:00:00+00:00,True
2,2014-12-09 00:00:00+00:00,2012-05-14 00:00:00+00:00,True
3,2014-12-26 00:00:00+00:00,2023-04-09 00:00:00+00:00,False
4,2015-01-13 00:00:00+00:00,2022-03-13 00:00:00+00:00,False
5,2015-01-13 00:00:00+00:00,2021-05-17 00:00:00+00:00,False
...,...,...,...
7382,2018-02-19 00:00:00+00:00,2022-04-06 00:00:00+00:00,False
7383,2018-02-19 00:00:00+00:00,2020-12-05 00:00:00+00:00,False
7384,2018-02-19 00:00:00+00:00,2020-12-18 00:00:00+00:00,False
7385,2018-02-19 00:00:00+00:00,2021-11-29 00:00:00+00:00,False


In [113]:
merged_airbnb_RNAL["license_date_after_review"].value_counts()

license_date_after_review
False    5562
True     1201
Name: count, dtype: int64

## Concentração

In [171]:
airbnb['host_listings_number'] = airbnb.groupby('host_id')['listing_url'].transform('count')
airbnb["host_more_than_1_listing"] = airbnb['host_listings_number'].apply(lambda x: x > 1)
airbnb["host_more_than_2_listings"] = airbnb['host_listings_number'].apply(lambda x: x > 2)
airbnb.groupby('host_id').size().value_counts()

1      1949
2       433
3       230
4       177
5       118
6        72
7        55
8        44
9        29
10       19
14       12
13       11
11       11
15       10
12        8
18        6
21        6
16        5
24        4
25        3
31        2
28        2
35        2
47        2
45        2
43        2
22        2
64        2
19        2
89        1
55        1
20        1
62        1
23        1
65        1
187       1
36        1
39        1
51        1
17        1
77        1
184       1
50        1
37        1
26        1
34        1
41        1
33        1
38        1
106       1
27        1
44        1
Name: count, dtype: int64

In [174]:
airbnb["host_more_than_1_listing"].value_counts()

host_more_than_1_listing
True     7646
False    1949
Name: count, dtype: int64

## Tipologia

In [176]:
airbnb["property_type"].value_counts()

property_type
Entire rental unit                    6356
Private room in rental unit            573
Entire serviced apartment              481
Entire loft                            391
Entire home                            339
Entire condo                           254
Private room in home                   242
Room in boutique hotel                 148
Private room in guesthouse             104
Room in hotel                           90
Entire townhouse                        56
Private room in bed and breakfast       52
Private room in hostel                  48
Private room in townhouse               44
Private room in villa                   43
Shared room in hostel                   32
Private room in guest suite             28
Shared room in rental unit              28
Room in aparthotel                      27
Entire guesthouse                       26
Private room in casa particular         25
Tiny home                               23
Entire vacation home                    

In [182]:
airbnb[airbnb["property_type"] == "Casa particular"].head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,license_cleaned,has_license,host_listings_number,host_more_than_1_listing,host_more_than_2_listings
1107,11523700,https://www.airbnb.com/rooms/11523700,20230612205726,2023-06-13,city scrape,casa particular in Porto · ★5.0 · 2 bedrooms ·...,Mus Boavista está situado numa zona privilegia...,"Na envolvente, pode encontrar a Casa da Música...",https://a0.muscache.com/pictures/ac4fb575-d3ef...,3743974,https://www.airbnb.com/users/show/3743974,Ricardo,2012-10-03,"Porto, Portugal",I'm a graphic designer from Porto working on a...,within an hour,100%,100%,f,https://a0.muscache.com/im/pictures/user/49cb4...,https://a0.muscache.com/im/pictures/user/49cb4...,NaN,1.0,1.0,"['phone', 'work_email']",t,t,"Porto, Porto District, Portugal",Lordelo do Ouro e Massarelos,PORTO,41.16046,-8.64486,Casa particular,Entire home/apt,4,NaN,1 bath,2.0,3.0,"[""Pack \u2019n play/Travel crib"", ""Paid parkin...",$142.00,3,1125,3,3,1125,1125,3.0,1125.0,NaN,t,11,37,38,38,2023-06-13,3,1,0,2022-03-06,2022-10-09,5.00,5.00,5.00,5.00,5.00,5.00,5.00,121243/AL,f,1,1,0,0,0.19,121243,RNAL,1,False,False
7626,49903150,https://www.airbnb.com/rooms/49903150,20230612205726,2023-06-13,city scrape,casa particular in Porto · ★4.17 · 1 bath,Situado perto do centro da cidade do Porto.<br...,NaN,https://a0.muscache.com/pictures/f17a0072-7520...,402413287,https://www.airbnb.com/users/show/402413287,Artur,2021-05-19,NaN,NaN,within an hour,100%,94%,f,https://a0.muscache.com/im/pictures/user/200ed...,https://a0.muscache.com/im/pictures/user/200ed...,NaN,1.0,2.0,"['email', 'phone']",t,f,NaN,"Cedofeita, Ildefonso, Sé, Miragaia, Nicolau, V...",PORTO,41.14985,-8.61668,Casa particular,Entire home/apt,2,NaN,1 bath,NaN,NaN,"[""Dishes and silverware"", ""Kitchen"", ""Cooking ...",$93.00,3,15,3,3,1125,1125,3.0,1125.0,NaN,t,30,60,90,180,2023-06-13,12,6,0,2022-04-03,2023-04-10,4.17,4.17,3.75,4.33,4.17,5.00,4.08,68371/AL,f,1,1,0,0,0.82,68371,None,1,False,False
7771,50678844,https://www.airbnb.com/rooms/50678844,20230612205726,2023-06-13,city scrape,casa particular in Porto · ★4.59 · 1 bedroom ·...,O Oporto Sao lazaro Apartment apresenta acomod...,NaN,https://a0.muscache.com/pictures/c84249ed-ddd9...,224645035,https://www.airbnb.com/users/show/224645035,Ana,2018-11-07,"Porto, Portugal","Olá, sou a Ana Oliveira e nasci na linda cidad...",within an hour,100%,99%,f,https://a0.muscache.com/im/pictures/user/798c5...,https://a0.muscache.com/im/pictures/user/798c5...,NaN,5.0,5.0,"['email', 'phone', 'work_email']",t,t,NaN,Bonfim,PORTO,41.14685,-8.59920,Casa particular,Entire home/apt,2,NaN,1 bath,1.0,1.0,"[""First aid kit"", ""Hair dryer"", ""Dining table""...",$55.00,2,7,2,4,7,1125,2.5,793.6,NaN,t,1,8,29,

## Sazonalidade

In [162]:
reviews = pd.read_csv("data/reviews.csv")
reviews = reviews[reviews["listing_id"].isin(set(airbnb["id"]))]
reviews['date'] = pd.to_datetime(reviews['date'])
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,41339,4619901,2013-05-15,200321,David,"Lovely bright, clean, quiet and airy apartment..."
1,41339,6550958,2013-08-16,1241699,Valentina,"Paula responsed very quickly, despite the fact..."
2,41339,10305825,2014-02-11,12007087,Carlos,Es muy amable y tiene mucha paciencia para los...
3,41339,15714205,2014-07-14,17261376,Anna,We loved our stay at Paula's apartment near th...
4,41339,73035636,2016-05-06,5443121,Yassine,"paula est une hote formidable, acceuillante et..."


In [154]:
reviews['date'] = pd.to_datetime(reviews['date'])
reviews['month'] = reviews['date'].dt.month
hotel_month_counts = reviews.groupby(['listing_id'])['month'].nunique().reset_index()
hotels_with_reviews_in_all_months = hotel_month_counts[hotel_month_counts['month'] >= 12]
num_hotels_with_reviews_in_all_months = len(hotels_with_reviews_in_all_months)
print("Number of hotels with reviews in all 12 months:", num_hotels_with_reviews_in_all_months)

Number of hotels with reviews in all 12 months: 3179


In [170]:
date_threshold = pd.Timestamp('2022-05-01')
filtered_reviews = reviews[reviews.groupby('listing_id')['date'].transform('min') <= date_threshold]
filtered_reviews['month'] = filtered_reviews['date'].dt.month
hotel_month_counts = filtered_reviews.groupby(['listing_id'])['month'].nunique().reset_index()
hotels_with_reviews_in_all_months = hotel_month_counts[hotel_month_counts['month'] >= 12]
num_hotels_with_reviews_in_all_months = len(hotels_with_reviews_in_all_months)
print("Number of hotels with reviews in all 12 months since May 2022:", num_hotels_with_reviews_in_all_months)

Number of hotels with reviews in all 12 months since May 2022: 3068


/var/folders/2_/dc69y37n7d14q5rbg8ly14g40000gn/T/ipykernel_30983/2730399199.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_reviews['month'] = filtered_reviews['date'].dt.month
